In [1]:
import os
import pandas as pd
import numpy as np
import re

In [2]:
#load data
data_folder = os.path.join(os.getcwd(), 'data')
category_query_csv = 'queryset_1178_fashion_v4.3.csv'
category_gt_csv = 'groundtruth_RESULT_1178_fashion_v4.3.csv'
category_query = pd.read_csv(os.path.join(data_folder, category_query_csv))
category_gt = pd.read_csv(os.path.join(data_folder, category_gt_csv))

In [3]:
category_query.head()

,Query Set URL,Category,Fq,Im_Name,Bounding Box,Box Categories,Tags,Customized Query Param,Ranking Tag,S3 URL,Image Id
0,https://s3-ap-southeast-1.amazonaws.com/visenz...,bag,NaN,NaN,"[[15,0,391,349]]",[bag],undefined:Bad Case,"{""detection"":""bag""}",NaN,https://visenze-productivity-evaluation.s3-acc...,2734638
1,https://s3-ap-southeast-1.amazonaws.com/visenz...,bag,NaN,NaN,"[[0,20,400,378]]",[bag],NaN,"{""detection"":""bag""}",NaN,https://visenze-productivity-evaluation.s3-acc...,2734639
2,https://s3-ap-southeast-1.amazonaws.com/visenz...,bag,NaN,NaN,"[[29,15,261,396]]",[bag],NaN,"{""detection"":""bag""}",NaN,https://visenze-productivity-evaluation.s3-acc...,2734640
3,https://s3-ap-southeast-1.amazonaws.com/visenz...,bag,NaN,NaN,"[[21,20,332,261]]",[bag],NaN,"{""detection"":""bag""}",NaN,https://visenze-productivity-evaluation.s3-acc...,2734641
4,https://s3-ap-southeast-1.amazonaws.com/visenz...,bag,NaN,NaN,"[[4,0,763,779]]",[bag],NaN,"{""detection"":""bag""}",NaN,https://visenze-productivity-evaluation.s3-acc...,2734642


In [4]:
category_gt.head()

,Query Set URL,Reference Set URL,Score,Im_Name,Category,S3 URL
0,https://s3-ap-southeast-1.amazonaws.com/visenz...,http://storage.googleapis.com/visenze-producti...,4.0,shoe_comments_ref_shoe_521496394262_main_1ae25...,shoe,https://visenze-productivity-evaluation.s3-acc...
1,https://s3-ap-southeast-1.amazonaws.com/visenz...,http://storage.googleapis.com/visenze-producti...,4.0,shoe_comments_ref_shoe_521496394262_main_332c7...,shoe,https://visenze-productivity-evaluation.s3-acc...
2,https://s3-ap-southeast-1.amazonaws.com/visenz...,http://storage.googleapis.com/visenze-producti...,3.0,2729eb88-5def-4efa-9cbe-8a11f5d6bc9d.jpg,eyewear,https://visenze-productivity-evaluation.s3-acc...
3,https://s3-ap-southeast-1.amazonaws.com/visenz...,http://storage.googleapis.com/visenze-producti...,4.0,spl5350_0.700x350.1437457549,eyewear,https://visenze-productivity-evaluation.s3-acc...
4,https://s3-ap-southeast-1.amazonaws.com/visenz...,http://storage.googleapis.com/visenze-producti...,4.0,70ring_jewelry_-1.jpg,jewelry,https://visenze-productivity-evaluation.s3-acc...


In [5]:
#length of unique Query Set URL in category_query is 3480
#length of unique Query Set URL in category_gt is 3277
#first delete all Query Set URL that does not have ground truth
query_set_urls_gt = set()
for url in category_gt['Query Set URL']:
    query_set_urls_gt.add(url)
query_set_urls = set()
idxes = []
for idx,url in enumerate(category_query['Query Set URL']):
    if url not in query_set_urls_gt:
        idxes.append(idx)
    elif url in query_set_urls:
        idxes.append(idx)
    else:
        query_set_urls.add(url)
print(str(len(idxes)) + ' rows in category_query are to be deleted because of eithor duplicate or do not have gt.')

232 rows in category_query are to be deleted because of eithor duplicate or do not have gt.


In [6]:
category_query_clean = category_query.drop(idxes)
category_query_clean.to_csv(os.path.join(data_folder, 'category_query_clean.csv'),index=False)

In [7]:
#split the query set by category
#the first query with category bag goes to dev bucket
#the second query with category bag goes to test bucket so on and so forth
label = dict()
for cate in category_query_clean['Category'].unique():
    label[cate] = 0
label_col = []
for idx, row in category_query_clean.iterrows():
    key = row['Category']
    label_col.append(label[key])
    label[key] = 1 - label[key]
category_query_clean['label'] = label_col

category_query_dev = category_query_clean.loc[category_query_clean['label'] == 0].drop(columns=['label'])
category_query_test = category_query_clean.loc[category_query_clean['label'] == 1].drop(columns=['label'])
category_query_dev.to_csv(os.path.join(data_folder, 'category_query_dev.csv'), index=False)
category_query_test.to_csv(os.path.join(data_folder, 'category_query_test.csv'), index=False)
print('query dev/test set saved to data folder.')

query dev/test set saved to data folder.


In [8]:
category_gt_dev = category_query_dev[['Query Set URL']].join(category_gt.set_index('Query Set URL'),on='Query Set URL',rsuffix='gt_')
category_gt_test = category_query_test[['Query Set URL']].join(category_gt.set_index('Query Set URL'),on='Query Set URL',rsuffix='gt_')
category_gt_dev.to_csv(os.path.join(data_folder, 'category_gt_dev.csv'), index=False)
category_gt_test.to_csv(os.path.join(data_folder, 'category_gt_test.csv'), index=False)
print('ground truth dev/test set saved to data folder.')

ground truth dev/test set saved to data folder.


In [9]:
#split reference set to dev/test
#remove duplicate of reference set
reference_set_csv = pd.read_csv(os.path.join(data_folder, 'reference_gcs.csv'))
reference_set_csv.drop_duplicates(subset=['im_url'],inplace=True)
reference_set_csv.reset_index(inplace=True)
reference_set_csv.drop(columns=['index'],inplace=True)

#give label (dev=0/test=1/other=2) to each row in reference_set
dev_gt = pd.DataFrame(category_gt_dev['Reference Set URL'].unique(), columns=['im_url'])
test_gt = pd.DataFrame(category_gt_test['Reference Set URL'].unique(), columns=['im_url'])
dev_test_gt = pd.concat([dev_gt, test_gt], ignore_index=True)
dev_test_gt.drop_duplicates(subset=['im_url'],inplace=True)
dev_test_gt['label'] = 0
reference_set = reference_set_csv.join(dev_test_gt.set_index('im_url'), on='im_url', rsuffix='gt_')

#filter and leave the images in the reference set that is not gt of dev/test set and split it into two
#dev_ref = half of the unallocate_ref_set + dev_gt
#test_ref = another half of the unallocate_ref_set + test_gt
unallocate_ref_set = reference_set.loc[reference_set['label'] != 0]
unallocate_ref_set = unallocate_ref_set.drop('label', axis=1)
dev_ref_set = unallocate_ref_set.sample(random_state=0,frac=0.5)
test_ref_set = unallocate_ref_set.drop(dev_ref_set.index)

#the function below is to prepare the part of ref set coming from gt
def prepare_ref_from_gt(ref_set):
    ref_set_from_gt = pd.DataFrame(columns=['im_name', 'im_url', 'desc', 'category'])
    
    ref_raw = ref_set[['Reference Set URL', 'Category']].drop_duplicates(subset=['Reference Set URL'])
    
    ref_set_from_gt['im_name'] = ref_raw['Reference Set URL'].apply(lambda row: row.split('/')[5])
    ref_set_from_gt['im_url'] = ref_raw['Reference Set URL']
    ref_set_from_gt['desc'] = ref_raw['Category']
    ref_set_from_gt['category'] = ref_raw['Category']
    return ref_set_from_gt

dev_ref_set_from_gt = prepare_ref_from_gt(category_gt_dev)
test_ref_set_from_gt = prepare_ref_from_gt(category_gt_test)

dev_ref_set = pd.concat([dev_ref_set, dev_ref_set_from_gt], ignore_index=True)
test_ref_set = pd.concat([test_ref_set, test_ref_set_from_gt], ignore_index=True)

In [10]:
# need to use samsung product tagging model to extract category
def convert_to_datapipeline(ref_set):
    ref_set_dp = pd.DataFrame(columns=['im_url', 'concepts', 'metadatas'])
    ref_set_dp['im_url'] = ref_set['im_url']
    ref_set_dp['concepts'] = "[{'model_name': None, 'version': None, 'labels': {'system': None, 'ground_truth_source': None, 'source_id': None}, 'objects': [{'tags': [{'tag_group': None, 'tag': None, 'full_tag': None, 'score': None}], 'box': None, 'score': None}], 'reference': []}, {'model_name': None, 'version': None, 'labels': {'system': None, 'ground_truth_source': None, 'source_id': None}, 'objects': [{'tags': [{'tag_group': None, 'full_tag': None, 'tag': None, 'score': None}], 'box': None, 'score': None}], 'reference': []}]"
    ref_set_dp['metadatas'] = '{}'
    return ref_set_dp
dev_ref_dp = convert_to_datapipeline(dev_ref_set)
test_ref_dp = convert_to_datapipeline(test_ref_set)
dev_ref_dp.to_csv(os.path.join(data_folder, 'dev_ref_dp.csv'), index=False)
test_ref_dp.to_csv(os.path.join(data_folder, 'test_ref_dp.csv'), index=False)

In [11]:
#process the predict result from datapipeline
dev_ref_predict = pd.read_csv(os.path.join(data_folder, 'dev_ref_set_category_predict.csv'))
test_ref_predict = pd.read_csv(os.path.join(data_folder, 'test_ref_set_category_predict.csv'))

def extract_predict(metadatas):
    match = re.search('(?<=\'root:)[a-z]+', metadatas)
    if match is not None:
        return match.group(0)
    return 'none'
    
dev_ref_predict['category'] = dev_ref_predict['metadatas'].apply(extract_predict)
dev_ref_predict.drop(columns=['concepts', 'metadatas'], inplace=True)
test_ref_predict['category'] = test_ref_predict['metadatas'].apply(extract_predict)
test_ref_predict.drop(columns=['concepts', 'metadatas'], inplace=True)

#add predict result to the ref set
def add_predict_to_ref(ref_set, ref_predict):
    ref_set.columns = ['im_name', 'im_url', 'desc', 'category_original']
    ref_set = ref_set.join(ref_predict.set_index('im_url'), on='im_url', rsuffix='_predict')
    ref_set = ref_set.reindex(columns=['im_name', 'im_url', 'category', 'desc', 'category_original'])
    return ref_set

dev_ref_set = add_predict_to_ref(dev_ref_set, dev_ref_predict)
test_ref_set = add_predict_to_ref(test_ref_set, test_ref_predict)
dev_ref_set.to_csv(os.path.join(data_folder, 'dev_ref_set.csv'), index=False)
test_ref_set.to_csv(os.path.join(data_folder, 'test_ref_set.csv'), index=False)

In [49]:
# process query dev/test dataset
dev_query_predict = pd.read_csv(os.path.join(data_folder, 'dev_query_predict.csv'))
test_query_predict = pd.read_csv(os.path.join(data_folder, 'test_query_predict.csv'))
dev_query_predict['category'] = dev_query_predict['metadatas'].apply(extract_predict)
dev_query_predict.drop(columns=['concepts', 'metadatas'], inplace=True)
dev_query_predict.drop_duplicates(subset=['im_url'],inplace=True)
test_query_predict['category'] = test_query_predict['metadatas'].apply(extract_predict)
test_query_predict.drop(columns=['concepts', 'metadatas'], inplace=True)
test_query_predict.drop_duplicates(subset=['im_url'],inplace=True)

dev_query_w_predict = category_query_dev.join(dev_query_predict.set_index('im_url'),on='S3 URL',rsuffix='_predict')
test_query_w_predict = category_query_test.join(test_query_predict.set_index('im_url'),on='S3 URL',rsuffix='_predict')
dev_query_w_predict.to_csv(os.path.join(data_folder, 'dev_query_w_predict.csv'), index=False)
test_query_w_predict.to_csv(os.path.join(data_folder, 'test_query_w_predict.csv'), index=False)